# Classification model using Decision Tree

### 1. Import libraries

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

### 2. Upload Clinic data

In [21]:
path ='../../../Data_preprocessing/Prediction PFS/Clinical_data_categorized_PFS.csv' 
data = pd.read_csv(path)
data.head()

,Unnamed: 0,RNA_ID,Age,Sarc,Rhab,Number_of_Prior_Therapies,Days_from_TumorSample_Collection_and_Start_of_Trial_Therapy,Tumor_Shrinkage,PFS,TM_TC_Ratio,...,MSKCC_NA,MSKCC_POOR,IMDC_FAVORABLE,IMDC_INTERMEDIATE,IMDC_NOT_REPORTED,IMDC_POOR,ImmunoPhenotype_Desert,ImmunoPhenotype_Excluded,ImmunoPhenotype_Infiltrated,ImmunoPhenotype_NA
0,0,NaN,73,0.0,0.0,2.0,808,0.0,1.545205,-1.0,...,0,0,0,0,1,0,0,0,1,0
1,1,NaN,55,0.0,0.0,2.0,1826,0.0,3.419178,-1.0,...,0,0,0,0,1,0,1,0,0,0
2,2,NaN,51,0.0,0.0,1.0,1541,0.0,1.413699,-1.0,...,0,0,0,0,1,0,0,0,1,0
3,3,NaN,70,0.0,0.0,3.0,-1,0.0,0.394521,-1.0,...,0,1,0,0,1,0,0,0,0,1
4,4,NaN,59,0.0,0.0,3.0,376,0.0,10.158904,-1.0,...,0,1,0,0,1,0,0,0,1,0


In [22]:
Y = [] # Target column
# For each entry I classified it by its PFS value.
for i in range (len(data)):
    if data.PFS[i]<3: # If PFS is lower than 3 months, I will consider it as NonResponder (NR)
        Y.append(0)
    elif data.PFS[i]<6: # If PFS is over 6 months, I will consider it as Responder (R)
        Y.append(1)
    else:
        Y.append(2) # If PFS is between 3 and 6 months, I will consider it as SemiResponder (SR)
        
data = data.drop('PFS', axis=1) # As we won't need this column any more, I deleted it.

X = data.iloc[:,2:26] 
# I selected all the columns by removing the Unnamed column (row id) and the Target column.
print(X)
print('Numero de pacientes: ',len(Y))

     Age  Sarc  Rhab  Number_of_Prior_Therapies  \
0     73   0.0   0.0                        2.0   
1     55   0.0   0.0                        2.0   
2     51   0.0   0.0                        1.0   
3     70   0.0   0.0                        3.0   
4     59   0.0   0.0                        3.0   
..   ...   ...   ...                        ...   
285   54   0.0   0.0                        3.0   
286   56   0.0   0.0                        2.0   
287   65   0.0   0.0                        2.0   
288   48   0.0   0.0                        1.0   
289   76   0.0   0.0                        2.0   

     Days_from_TumorSample_Collection_and_Start_of_Trial_Therapy  \
0                                                  808             
1                                                 1826             
2                                                 1541             
3                                                   -1             
4                                              

### 3. Train-Test dataset split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=125)


print('Training set size:', len(X_train))
print('Target column size of the training set:', len(y_train))
print('Test set size:', len(X_test))
print('Target column size of the test set:', len(y_test))

Training set size: 232
Target column size of the training set: 232
Test set size: 58
Target column size of the test set: 58


### 4. Select the parameters of the model and fit it

In [24]:
param_grid = {'C': [1,2,5,7,10],
            'solver': ['liblinear'], # No escogemos ‘lbfgs’, ‘sag’ ni ‘saga’ porque no termina de 
                                                 # ejecutarse debido a: "TOTAL NO. of ITERATIONS REACHED LIMIT"
            'max_iter':[25,50,100,200,500,1000],
            'random_state':[125]}

# I created a GridSearchCV which allows us to systematically evaluate and select the parameters of our model.
# By indicating a model and the parameters to test, you can evaluate the performance of the first one based on the
# seconds through cross validation.
clf = GridSearchCV(LogisticRegression(), param_grid, cv = 5)
clf.fit(X_train , y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [1, 2, 5, 7, 10],
                         'max_iter': [25, 50, 100, 200, 500, 1000],
                         'random_state': [125], 'solver': ['liblinear']})

In [25]:
print("Best estimate of parameters according to GridSearchCV:")
model = clf.best_estimator_
# Fit the model with the best parameters
model.fit(X_train , y_train)

Best estimate of parameters according to GridSearchCV:


LogisticRegression(C=2, max_iter=25, random_state=125, solver='liblinear')

In [26]:
print("Best result of the cross validation of the model with the best paramters:" +str(clf.best_score_))

Best result of the cross validation of the model with the best paramters:0.4225716928769658


### 5. Prediction

In [27]:
# Making predictions with the optimal model on the training dataset
yhatTrain = model.predict(X_train)
contTrain = 0

# Comparing with the Target column and check how many hits there have been
for i in range(0,len(y_train),1) :
    if (yhatTrain[i] == y_train[i]):
        contTrain = contTrain + 1

In [28]:
# Making predictions with the optimal model on the test dataset
yhatTest = model.predict(X_test)
contTest = 0

# Comparing with the Target column and check how many hits there have been
for i in range(0,len(y_test),1) :
    if (yhatTest[i] == y_test[i]):
        contTest = contTest + 1

In [29]:
print('Final accuracy on the training dataset:' + str(contTrain/len(y_train)))
print('Final accuracy on the testing dataset: ' + str(contTest/len(y_test)))

Final accuracy on the training dataset:0.5
Final accuracy on the testing dataset: 0.4827586206896552


In [30]:
from sklearn.metrics import classification_report,confusion_matrix

print('----------------Confusion Matrix (Training)------------------')
print(confusion_matrix(y_train,yhatTrain))
print('Input data:  ' + str(np.array(y_train)))
print('Prediction:        ' +str(yhatTrain))

----------------Confusion Matrix (Training)------------------
[[78  6 21]
 [33  8 13]
 [41  2 30]]
Input data:  [2 0 1 2 0 2 1 2 2 1 1 0 1 0 2 2 2 2 1 2 1 2 1 0 0 2 1 1 0 0 0 2 0 0 1 2 2
 2 0 0 2 0 2 2 0 2 1 2 0 0 2 0 2 0 0 0 0 0 1 0 2 1 2 0 2 0 2 2 2 2 2 2 0 2
 1 2 0 0 0 0 2 0 0 2 2 2 0 0 1 1 1 2 2 1 1 2 0 0 0 0 2 2 1 0 0 0 1 1 2 2 2
 2 0 0 2 0 0 0 0 2 0 1 2 1 2 2 0 1 0 0 0 0 0 0 0 1 0 2 2 1 0 0 0 0 2 2 2 0
 1 1 0 2 0 2 2 2 0 0 0 0 1 1 2 0 2 0 0 1 0 0 0 1 2 0 2 1 2 0 2 0 0 0 1 2 1
 1 0 1 0 1 1 1 0 0 2 1 0 2 0 1 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 2 1 1
 0 2 0 0 2 0 1 1 0 0]
Prediction:        [0 0 2 0 0 2 1 0 0 0 2 0 0 2 0 0 2 2 0 2 0 2 1 2 0 0 0 2 0 2 1 2 0 0 0 0 0
 0 1 0 0 0 0 0 0 2 0 2 2 0 0 0 0 2 1 0 0 2 0 0 0 0 0 2 0 0 0 2 0 2 0 0 0 2
 0 0 0 0 0 2 2 0 2 0 0 2 0 0 0 2 0 0 0 1 0 1 2 0 0 1 0 0 0 0 0 0 0 0 0 0 2
 2 0 0 2 0 0 0 0 0 0 2 2 0 2 0 2 1 0 0 0 0 0 2 2 1 2 2 1 0 2 0 0 2 0 0 0 0
 0 1 0 2 2 0 0 2 2 0 1 0 0 0 0 0 0 0 0 2 2 0 0 0 2 0 2 0 0 0 2 0 0 0 2 2 0
 0 0 0 2 0 0 0 0 0 2 2

In [31]:
print(classification_report(y_train,yhatTrain))

              precision    recall  f1-score   support

           0       0.51      0.74      0.61       105
           1       0.50      0.15      0.23        54
           2       0.47      0.41      0.44        73

    accuracy                           0.50       232
   macro avg       0.49      0.43      0.42       232
weighted avg       0.50      0.50      0.47       232



In [32]:
print('----------------Confusion Matrix (Test)------------------')
print(confusion_matrix(y_test,yhatTest))
print('Input data:  ' + str(np.array(y_test)))
print('Prediction:        ' +str(yhatTest))

----------------Confusion Matrix (Test)------------------
[[20  3  2]
 [13  0  2]
 [ 8  2  8]]
Input data:  [2 2 1 2 0 1 1 2 2 2 0 0 2 0 1 0 0 2 0 2 2 0 2 2 1 0 0 1 1 0 0 2 1 0 0 0 1
 1 1 1 2 2 0 1 0 0 0 0 1 0 2 0 2 1 0 2 0 0]
Prediction:        [2 2 2 0 0 0 0 0 2 2 0 0 2 0 0 0 1 0 0 2 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 2 2 2 2 0 0 0 0 2 0 1 0 0 0 0 0 1 0 0]


In [33]:
print(classification_report(y_test, yhatTest))

              precision    recall  f1-score   support

           0       0.49      0.80      0.61        25
           1       0.00      0.00      0.00        15
           2       0.67      0.44      0.53        18

    accuracy                           0.48        58
   macro avg       0.38      0.41      0.38        58
weighted avg       0.42      0.48      0.43        58



*Everolimus test*

In [34]:
path ='../../../Data_preprocessing/Testing with everolimus/Clinical_data_categorized_everolimus.csv'
data = pd.read_csv(path)

Y = [] # Target column
# For each entry I classified it by its PFS value.
for i in range (len(data)):
    if data.PFS[i]<3: # If PFS is lower than 3 months, I will consider it as NonResponder (NR)
        Y.append(0)
    elif data.PFS[i]<6:
        Y.append(1)
    else:
        Y.append(1)# If PFS is over 3 months, I will consider it as Responder (R)
data=data.drop(['Unnamed: 0', 'PFS', 'RNA_ID'], axis=1)

In [35]:
len(data.columns)

23

In [36]:
# Making predictions with the optimal model on the test dataset
yhatTest = model.predict(data)
contTest = 0

# Comparing with the Target column and check how many hits there have been
for i in range(0,len(Y),1) :
    if (yhatTest[i] == Y[i]):
        contTest = contTest + 1

print('Final accuracy on the testing dataset: ' + str(contTest/len(Y)))

Final accuracy on the testing dataset: 0.3333333333333333


In [37]:
print('----------------Confusion Matrix (Test)------------------')
print(confusion_matrix(Y,yhatTest))
print('Input data:  ' + str(np.array(Y)))
print('Prediction:        ' +str(yhatTest))
print(classification_report(Y,yhatTest))

----------------Confusion Matrix (Test)------------------
[[44  2 19]
 [55 12 36]
 [ 0  0  0]]
Input data:  [1 1 0 1 1 0 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 0 1 0 1 0 1 1 0 1 1 1 1 0 0 1
 1 0 0 1 1 1 1 0 0 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 0 1 0 0 0 1 1 1 1 1 1 1 1
 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0
 1 0 0 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 0 1 0 1 1 1 0 0 1 0 1 0 1 1 1 0 0]
Prediction:        [0 2 2 1 2 0 0 2 0 0 0 0 0 0 0 0 0 1 0 2 0 2 0 0 0 2 0 0 2 2 2 2 0 0 0 2 1
 0 2 0 0 0 0 0 0 0 0 0 2 0 0 2 2 2 0 1 2 1 0 0 2 2 2 1 2 2 0 2 0 2 0 2 1 0
 0 0 1 0 0 2 0 2 2 2 0 0 0 2 2 0 0 2 1 2 0 0 1 0 0 0 0 0 2 0 0 0 0 0 0 0 2
 0 0 2 2 2 0 0 0 2 0 0 2 1 0 2 2 2 2 0 0 0 2 2 0 0 0 2 0 0 0 0 2 2 0 2 0 1
 0 0 1 0 0 0 0 1 0 2 0 2 0 2 0 2 0 0 0 0]
              precision    recall  f1-score   support

           0       0.44      0.68      0.54        65
           1       0.86      0.12      0.21       103
           2       0.00   

C:\Users\sandr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sandr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sandr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
